In [1]:
from collections import defaultdict
import re
import pickle
import os
import random
import pickle
import numpy as np

## Shard doordash data by category

This is basically a manual map-reduce. First for each shard, we separate by the category name, then we gather entries across shards by category.

In [2]:
def get_by_category(shard):
    datafile = '/home/doordash/doordash_{}.pkl'.format(shard)
    with open(datafile, 'rb') as f:
        print("Opened {}".format(datafile))
        json = pickle.load(f)
        all_entries = []
        for entry in json:
            title = entry['title']
            section = entry['metadata']['section']
            description = entry['metadata']['description']
            category = entry['metadata'].get('restaurant_category', None)
            all_entries.append((title, description, section, category))
    print("Loaded all entries")
    category_to_entries = defaultdict(list)
    for entry in all_entries:
        og_category = entry[3]
        if og_category is None:
            og_category = 'None'
        if og_category == '':
            og_category = 'blank'
        format_category = re.sub(r"\s+", '-', og_category)
        format_category = re.sub(r"\&", "and", format_category)
        format_category = format_category.lower()
        # (title, description) in a list
        category_to_entries[format_category].append((entry[0], entry[1]))
    print("Made dictionary")
    for category, entries in category_to_entries.items():
        print(category, len(entries))
        with open(
            '/home/doordash-sharded/doordash_{}_{}'.format(category, shard), 'wb') as f:
            pickle.dump(entries, f)

In [3]:
for i in range(0, 16):
    get_by_category(i)

Opened /home/doordash/doordash_0.pkl
Loaded all entries
Made dictionary
fast-food 25435


FileNotFoundError: [Errno 2] No such file or directory: '/home/doordash-sharded/doordash_fast-food_0'

In [ ]:
category_to_filenames = defaultdict(list)
rootdir = '/home/doordash-sharded'
for filename in os.listdir(rootdir):
    _, category, shard = filename.split("_")
    category_to_filenames[category].append(
        os.path.join(rootdir, filename))

for category, filenames in category_to_filenames.items():
    all_items = []
    for filename in filenames:
        with open(filename, 'rb') as f:
            all_items.extend(pickle.load(f))
    with open('/home/doordash-by-category/{}'.format(category), 'wb') as f:
        pickle.dump(all_items, f)

In [ ]:
from collections import Counter
category_items_counter = Counter()
for filename in os.listdir('/home/doordash-by-category'):
    with open(os.path.join('/home/doordash-by-category', filename),
              'rb') as f:
        all_items = pickle.load(f)
        category_items_counter[filename] = len(all_items)

In [ ]:
print(category_items_counter.most_common())

# Take sharded data and clean

In [515]:
cuisine = 'other'
with open(os.path.join('/home/doordash-by-dish', cuisine),
              'rb') as f:
    all_items = pickle.load(f)
    all_titles = [e[0] for e in all_items]

In [516]:
re.sub("[\(\[].*?[\)\]]", "", 'Bacon & Sausage (2 Each)')
re.sub(r"\&", "and", "Bacon & Sausage (2 Each)")

'Bacon and Sausage (2 Each)'

In [520]:
random.choices(all_items, k=10)

[('Oreo McFlurry', '(340 Cal.)'),
 ('Terra Mare',
  'Chicken, shrimp, broccoli, roasted pepper, fresh tomato, pesto sauce.'),
 ('Strawberry Shake', ''),
 ('Three Village', ''),
 ('71. Chinese Herbal Tea', ''),
 ('Beef w. String Beans (Combo Platter)', ''),
 ('Jarritos', ''),
 ('Cat Fish Wrap Combo', 'Served with fries and 16 oz. drink.'),
 ('36e. Puri', 'Whole wheat puffy bread.'),
 ('Fettuccini Alfredo', 'A Rich Parmesan Cream Sauce')]

In [356]:
def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [357]:
def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

In [358]:
letter_items = [format_valid(x[0]) for x in all_items if check_valid(x[0])]

In [361]:
len([x for x in letter_items if x==" "])

0

In [312]:
len(letter_items), len(all_items)

(1137599, 1562955)

In [ ]:
random.choices(letter_items, k=10)

## Take sharded data and tokenize

In [113]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=True)

##### Demonstrate with particular cuisine

In [5]:
cuisine = 'soup'
with open(os.path.join('/home/doordash-by-dish', cuisine),
              'rb') as f:
    all_items = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/sarahwooders_gmail_com/transformers/doordash-tmp/healthy'

In [524]:
titles = [(e[0], e[1]) for e in all_items]
print(titles[:100])

[('Mac & Cheese', ''), ('French Onion', 'Available monday, wednesday, friday, sunday. Served with our fresh baked baguette.'), ('Beef Chili', 'Available monday, wednesday and friday. Served with our fresh baked baguette.'), ('Chicken Chili', 'Available tuesday, thursday, saturday, and sunday. Served with our fresh baked baguette.'), ('Clam Chowder', 'Available friday. Served with our fresh baked baguette.'), ('Broccoli Cheese', 'Available Everyday! Served with a fresh baguette.'), ('Chicken Noodle', 'Available tuesday, thursday, saturday.'), ('Fire Roasted Vegetable', 'Available monday, and wednesday.'), ('Italian Wedding', 'Available wednesday, saturday.'), ('Golden Lentil', 'Available tuesday, thursday, saturday.'), ('Butternut Squash', 'Available Everyday!'), ('Tempura Udon', 'Udon noodle, seasonal vegetables in clear broth, served with tempura shrimp and vegetable on the side.'), ('Seafood Udon', 'Shrimp, scallop, crab meat, seafood cake, seasonal vegetables in clear broth.'), ('To

In [435]:
max_seq_length=64
tokenized_output = tokenizer.batch_encode_plus(
    titles[:5], max_length=max_seq_length,
    pad_to_max_length=False)
print(tokenized_output)

{'input_ids': [[101, 19868, 11731, 10419, 16080, 102, 10464, 16080, 11731, 10419, 20442, 30865, 10108, 58768, 103189, 102], [101, 19868, 30231, 13326, 102, 102], [101, 19868, 36833, 11268, 102, 102], [101, 19868, 10201, 47263, 58286, 102, 102], [101, 19868, 11731, 10419, 10142, 10105, 11795, 119, 102, 10464, 59227, 10123, 10108, 11731, 10419, 20442, 30865, 10108, 58768, 103189, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


###### Getting max seq length

In [436]:
cuisine_to_lens = {}
for cuisine in os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/doordash-by-category/{}'.format(cuisine)):
        continue
    print(cuisine)
    with open(os.path.join('/home/doordash-by-category', cuisine),
                  'rb') as f:
        all_items = pickle.load(f)
    # titles = [e[0] for e in all_items]
    titles = all_items
    random.shuffle(titles)
    tokenized_output = tokenizer.batch_encode_plus(
        titles[:1000], max_seq_length=128,
        pad_to_max_length=False)
    lens = [len(input_ids) for input_ids in tokenized_output['input_ids']]
    cuisine_to_lens[cuisine] = lens

sandwiches
deals
ethiopian
pickup
soup
smoothie
good-for-groups
alcohol-stores
beauty
exclusive
tapas
thai
german
cafes
pho
alcohol
belgian
indian
mediterranean
seafood
vietnamese
catering
mexican
greek
comfort-food
irish
european
ramen
heritage
bakery
newly-added
french
pakistani
takeout
desserts
halal
chicken
bubble-tea
drinks
kwb
fondue
argentine
steak
pasta
cheese
italian
late-night
japanese
south-asian
chinese
peruvian
asian
breakfast
coffee-and-tea
southern
healthy
other
cajun
brazilian
lunch
russian
none
pizza
delis
korean
barbecue
vegetarian
grocery
caribbean
noodles
filipino
british
gastropubs
burmese
sushi
middle-eastern
vegan
blank
latin-american
convenience
poke
poutineries
coffee
kosher
convenience-exclusive
gluten-free
african
burgers
salad
fast-food
food-and-drink
american
australian
spanish


In [438]:
{c: (max(l), np.mean(l)) for c, l in cuisine_to_lens.items()}

{'sandwiches': (131, 31.148),
 'deals': (90, 20.979),
 'ethiopian': (158, 26.525),
 'pickup': (207, 23.275),
 'soup': (141, 31.601),
 'smoothie': (178, 22.119),
 'good-for-groups': (75, 28.458),
 'alcohol-stores': (62, 19.620512820512822),
 'beauty': (104, 27.714285714285715),
 'exclusive': (20, 8.80327868852459),
 'tapas': (119, 25.598),
 'thai': (248, 24.955),
 'german': (103, 22.418),
 'cafes': (211, 23.737),
 'pho': (207, 24.777),
 'alcohol': (133, 28.292),
 'belgian': (92, 26.506214689265537),
 'indian': (112, 23.71),
 'mediterranean': (112, 21.779),
 'seafood': (105, 22.939),
 'vietnamese': (118, 22.937),
 'catering': (127, 25.21),
 'mexican': (139, 23.006),
 'greek': (122, 22.752),
 'comfort-food': (123, 25.192),
 'irish': (101, 26.843),
 'european': (148, 21.012),
 'ramen': (139, 23.536),
 'heritage': (51, 23.4375),
 'bakery': (192, 21.538),
 'newly-added': (90, 24.66793893129771),
 'french': (112, 23.863),
 'pakistani': (263, 21.227),
 'takeout': (89, 19.767),
 'desserts': (25

### Saving tokenization

In [571]:
rootdir = '/home/doordash-by-dish-no-and/title-tokenized/bert-base-multilingual-cased'
max_seq_length = 64
assert os.path.exists(rootdir)

In [572]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"

In [573]:
do_clean = True

def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [574]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'alcohol', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french',
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}
all_categories = [item for sublist in targets_to_categories.values() for item in sublist]

In [575]:
all_categories = [
    'burger', 'other', 'pizza', 'salad', 'sandwich', 'soup', 'sushi']

In [576]:
print(max_seq_length)

64


In [577]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"
lens = {}
for cuisine in all_categories:# os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/doordash-by-dish-no-and/{}'.format(cuisine)):
        # Not a real cuisine, for example bert-base-multilingual-cased is a folder containing tokenizations
        continue
    with open(os.path.join('/home/doordash-by-dish-no-and', cuisine), 'rb') as f:
        all_items = pickle.load(f)
    titles = [e[0] for e in all_items]
    if do_clean:
        titles = [format_valid(t) for t in titles if check_valid(t)]
        # titles = [(format_valid(t[0]), t[1]) for t in titles if check_valid(t[0])]
    lens[cuisine] = len(titles)
#     if os.path.exists(os.path.join(
#         rootdir, file_pattern.format(
#             cuisine=cuisine, max_seq_len=max_seq_length, 
#             input_key="input_ids"))):
#         # Tokenziation already exists in the given folder
#         continue

    print(cuisine, len(titles))
    tokenized_output = tokenizer.batch_encode_plus(
        titles,
        max_length=max_seq_length,
        pad_to_max_length=True)
    
    print("Done tokenizing")
    for input_key, arr in tokenized_output.items():
        filename = os.path.join(rootdir,
                                file_pattern.format(cuisine=cuisine,
                                      max_seq_len=max_seq_length,
                                      input_key=input_key))
        arr = np.array(arr)
        assert arr.shape == (len(titles), max_seq_length)
        fp = np.memmap(filename, dtype='int64', mode='w+',
                      shape=arr.shape)
        fp[:] = arr[:]
        del fp

burger 150367
Done tokenizing
other 2177797
Done tokenizing
pizza 252880
Done tokenizing
salad 366643
Done tokenizing
sandwich 414634
Done tokenizing
soup 123072
Done tokenizing
sushi 152763
Done tokenizing


In [579]:
with open(os.path.join(rootdir, 'lens.pkl'), 'wb') as f:
    # Keep lens which is needed for loading a memmap
    pickle.dump(lens, f)

# Dataloader

In [580]:
import torch
import itertools
from collections import Counter, OrderedDict, defaultdict
from torch.utils.data import DataLoader

In [581]:
class IterableTitles(torch.utils.data.IterableDataset):
    def __init__(self, cuisine, root_dir, max_seq_length, len_dict):
        super(IterableTitles).__init__()
        self.cuisine = cuisine
        self.max_seq_length = max_seq_length
        self.datafile_pattern = os.path.join(
            root_dir, 
            'titles_{}_{}_{{}}'.format(cuisine, max_seq_length))
        self.shape = (len_dict[cuisine], max_seq_length)
        self.length = len_dict[cuisine]

    def __len__(self):
        return self.length

    def __iter__(self):
        input_key_to_memmap = {}
        for input_key in ['input_ids',
                          'attention_mask',
                         'token_type_ids']:
            datafile = self.datafile_pattern.format(input_key)
            fp = np.memmap(
                datafile, dtype='int64', mode='r+',
                shape=self.shape)
            input_key_to_memmap[input_key] = fp
        while True:
            i = random.choice(range(self.shape[0]))
        #for i in itertools.cycle(range(self.shape[0])):
            yield (
                input_key_to_memmap['input_ids'][i],
                input_key_to_memmap['token_type_ids'][i],
                input_key_to_memmap['attention_mask'][i])


class MultiStreamDataLoader:
    def __init__(self, targets_to_categories, batch_size, rootdir):
        self.targets = list(sorted(targets_to_categories.keys()))
        self.targets_to_categories = targets_to_categories
        
        self.categories_to_target = OrderedDict()
        self.targets_to_categories_weights= {}
        self.categories_to_dataset = {}
        self.categories_to_dataset_iter = {}
        with open(os.path.join(rootdir, 'lens.pkl'), 'rb') as f:
            self.categories_to_len = pickle.load(f)
        
        for t, list_of_c in targets_to_categories.items():
            for c in list_of_c:
                self.categories_to_target[c] = t
                dataset = IterableTitles(c, rootdir, 64, self.categories_to_len)
                self.categories_to_dataset[c] = dataset
                self.categories_to_dataset_iter[c] = iter(DataLoader(dataset, batch_size=None))

        for t, c_list in self.targets_to_categories.items():
            self.targets_to_categories_weights[t] = [self.categories_to_len[c] for c in c_list]

        self.batch_size = batch_size
        self.total_samples = sum(self.categories_to_len.values())


    def __len__(self):
        return self.total_samples//self.batch_size
    

    def __iter__(self):
        while True:
            buffer = []
            labels = []
            target_choices = random.choices(self.targets, k=self.batch_size)
            category_choices = []
            for t in target_choices:
                category_choices.append(random.choices(
                    self.targets_to_categories[t],
                    weights=self.targets_to_categories_weights[t],
                    k=1)[0])

            category_counter = Counter(category_choices)
            category_labels = []
            for category, num_sample in category_counter.items():
                category_labels.extend([category for _ in range(num_sample)])
                l_num = self.targets.index(self.categories_to_target[category])
                labels.extend([l_num for _ in range(num_sample)])
                buffer.extend(
                    [next(self.categories_to_dataset_iter[category]) for _ in range(num_sample)]
                )
            yield (torch.stack([b[0] for b in buffer]),
                   torch.stack([b[1] for b in buffer]),
                   torch.stack([b[2] for b in buffer]),
                   torch.tensor(labels),
                   category_labels
                  )

In [582]:
# rootdir = '/home/doordash-by-category/cleaned-tokenized/bert-base-multilingual-cased/'
# rootdir = '/home/doordash-by-category/title-desc-tokenized/bert-base-multilingual-cased/'
rootdir = '/home/doordash-by-dish-no-and/title-tokenized/bert-base-multilingual-cased'



In [583]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', # 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french', # 'alcohol'
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}
targets_to_categories = {
    'burger': ['burger'],
    'other': ['other'],
    'pizza': ['pizza'],
    'salad': ['salad'],
    'sandwich': ['sandwich'],
    'soup': ['soup'],
    'sushi': ['sushi']
}
ds = MultiStreamDataLoader(targets_to_categories,
                           batch_size=800,
                           rootdir=rootdir)

In [584]:
ds.targets

['burger', 'other', 'pizza', 'salad', 'sandwich', 'soup', 'sushi']

In [585]:
# ds.targets_to_categories_weights

In [586]:
[(t, sum(w)) for t, w in ds.targets_to_categories_weights.items()]

[('burger', 150367),
 ('other', 2177797),
 ('pizza', 252880),
 ('salad', 366643),
 ('sandwich', 414634),
 ('soup', 123072),
 ('sushi', 152763)]

In [587]:
for batch_num, batch in enumerate(ds):
    if batch_num == 5:
        break
    print(Counter(batch[3].numpy()))
    input_ids = batch[0]
    labels = batch[3]
    categories = batch[4]
    for i, ids in enumerate(input_ids):
        print(
            ds.targets[labels[i]], "({})".format(categories[i]),
            ":",
            tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True)),
            # "\n",
            # ids
        )
    print("=====")

Counter({4: 125, 6: 122, 5: 115, 0: 113, 3: 112, 2: 111, 1: 102})
soup (soup) : mushrooms soup
soup (soup) : chicken soup
soup (soup) : tom sab brisket beef
soup (soup) : osumashi soup
soup (soup) : double romanian tenderloin soup
soup (soup) : egg drop soup
soup (soup) : egg drop soup
soup (soup) : tom kha goong
soup (soup) : cream of mushroom
soup (soup) : chicken soup
soup (soup) : tomato basil
soup (soup) : wonton egg drop soup
soup (soup) : chicken noodle soup
soup (soup) : curry red lentil
soup (soup) : tom yum noodle soup
soup (soup) : wonton soup
soup (soup) : chicken soup
soup (soup) : john bon
soup (soup) : haleem
soup (soup) : calo de res
soup (soup) : chicken
soup (soup) : minestrone
soup (soup) : american soup
soup (soup) : chicken tortilla
soup (soup) : broccoli and cheddar soup
soup (soup) : conch
soup (soup) : chicken pepper soup
soup (soup) : aguadito de camarones
soup (soup) : sopa de mariscos
soup (soup) : chicken noodle
soup (soup) : baked potato cheddar soup
soup (

pizza (pizza) : very very veggie pizza
pizza (pizza) : pepperoni
pizza (pizza) : pizza by the slice
pizza (pizza) : medium eggplant florentine pizza
pizza (pizza) : gourmet pizzas small
pizza (pizza) : bbq chicken
pizza (pizza) : ranch chicken pizza
pizza (pizza) : diamond head mini
pizza (pizza) : veggie pizza
pizza (pizza) : sm chicken calzone
pizza (pizza) : great white
pizza (pizza) : meat lovers
pizza (pizza) : cheese
pizza (pizza) : green goat
pizza (pizza) : buffalo chicken pizza
pizza (pizza) : calzone
pizza (pizza) : philly steak foldover
pizza (pizza) : bbq chicken
pizza (pizza) : supreme
pizza (pizza) : crispino pizza
pizza (pizza) : large meat sauce pizza
pizza (pizza) : cheese
pizza (pizza) : testaccio double
pizza (pizza) : five cheese pizza
pizza (pizza) : the bronx
pizza (pizza) : fiery buffalo chicken
pizza (pizza) : bbq chicken pizza
pizza (pizza) : sheetpan
pizza (pizza) : pizza
pizza (pizza) : chicken bacon stuffed baking required
pizza (pizza) : jimi
pizza (pizza) 

burger (burger) : bacon cheeseburger
burger (burger) : cheese burger
burger (burger) : chili cheese burger
burger (burger) : little hamburger
burger (burger) : gut buster
burger (burger) : california
burger (burger) : bacon cheeseburger
burger (burger) : fry lovers burger
burger (burger) : cheeseburger
sushi (sushi) : oshinko
sushi (sushi) : squid
sushi (sushi) : pepper tuna tataki
sushi (sushi) : chicken teriyaki roll
sushi (sushi) : sushi deluxe combo
sushi (sushi) : mango paradise
sushi (sushi) : deluxe roll
sushi (sushi) : dragon roll
sushi (sushi) : cali roll
sushi (sushi) : boston roll
sushi (sushi) : spicy tuna combo
sushi (sushi) : saba
sushi (sushi) : inari tofu skin
sushi (sushi) : sashimi assortment
sushi (sushi) : super crunchy roll
sushi (sushi) : albacor
sushi (sushi) : salmon skin
sushi (sushi) : rainbow roll
sushi (sushi) : mackerel
sushi (sushi) : sashimi regular
sushi (sushi) : salmon
sushi (sushi) : yellowtail
sushi (sushi) : tokyo roll
sushi (sushi) : tuna salmon ro

soup (soup) : thai soup special
soup (soup) : vegetable bean curd soup
soup (soup) : loaded potato
soup (soup) : brunswick stew
soup (soup) : fish fillet with pickled vegetable soup
soup (soup) : roast pork with vegetables noodle soup
soup (soup) : egg drop soup
soup (soup) : crock of french onion soup
soup (soup) : pho nam gan
soup (soup) : seafood soup
soup (soup) : miso soup
soup (soup) : small soup
soup (soup) : chicken wonton soup
soup (soup) : garden vegetable
soup (soup) : femented rice soup with mochi
soup (soup) : wonton soup
soup (soup) : tomato soup
soup (soup) : lentil soup
soup (soup) : chicken noodle
soup (soup) : egg drop soup
soup (soup) : tom yum soup
soup (soup) : noodle with shrimp
soup (soup) : chicken noodle soup
soup (soup) : clear chicken
soup (soup) : pho dac biet
soup (soup) : tomato soup
soup (soup) : wonton soup
soup (soup) : wonton soup
soup (soup) : minestrone
soup (soup) : wonton soup
soup (soup) : red peas
soup (soup) : ca ri ga
soup (soup) : sizzling ric

other (other) : chocolate milk
other (other) : dirty chai
other (other) : cheese
other (other) : unsweetened tea
other (other) : mocha
other (other) : chocolate phosphate
other (other) : bowl of loaded potato
other (other) : almond biscotti
other (other) : bc honey q
other (other) : meatballs with marinara
other (other) : soft drinks
other (other) : sprite
other (other) : seafood clear noodles
other (other) : dasani sparkling mixed berry
other (other) : tater tots
other (other) : bacon slice
other (other) : three cheese chicken quesadilla
other (other) : yogurt muffins
other (other) : mixed fry curry
other (other) : sweet berry cakes
other (other) : malta hatuey
other (other) : garlic prawn
other (other) : bolognese
other (other) : egg
other (other) : boneless
other (other) : pandan lemonade
other (other) : sweet potato pie
other (other) : frozen lemonade
other (other) : bread pudding
other (other) : fried green plantains
other (other) : chicken kebab
other (other) : corn tortillas
oth

...

soup (soup) : chicken cream corn soup
soup (soup) : chicken rice
soup (soup) : udon with shrimp tempura
soup (soup) : miso soup
soup (soup) : chicken rice soup
soup (soup) : hot soups
soup (soup) : soft tofu with vegetable soup
soup (soup) : tom yum shrimp
soup (soup) : homemade vegetable soup
soup (soup) : vegetarian daily soup special
soup (soup) : lentils
soup (soup) : pho ga
soup (soup) : chicken rice soup
soup (soup) : chicken noodle soup
soup (soup) : diablo shrimp tequila
soup (soup) : camaron
soup (soup) : fish roe soup
soup (soup) : seafood hot and sour soup
soup (soup) : potato soup
soup (soup) : fish noodle soup
soup (soup) : seafood udon
soup (soup) : tom yum soup
soup (soup) : pho nam gau gan
soup (soup) : bowl of ox tail tomato soup
soup (soup) : veggie soup with tofu
soup (soup) : tom yum
soup (soup) : chicken rice noodle soup
soup (soup) : duk gook
soup (soup) : greek lemon chicken soup
soup (soup) : pasta fagioli soup
soup (soup) : spicy beef soup
other (other) : class

burger (burger) : bbq burger
burger (burger) : build your own burger
burger (burger) : western bbq burger
burger (burger) : gyro burger
burger (burger) : southwestern black bean burger
burger (burger) : quesadilla burger
burger (burger) : turkey burger
burger (burger) : quarter pound king
burger (burger) : chipotle heat burger
burger (burger) : beef burger
burger (burger) : conan
burger (burger) : whopper with cheese
burger (burger) : old south
burger (burger) : frisco burger
burger (burger) : the better with cheddar
burger (burger) : tex mex burger
burger (burger) : bacon all american cheeseburger
burger (burger) : classic veggie burger
burger (burger) : premium veggie burger
burger (burger) : cheeseburger sub
burger (burger) : philly cheese moo
burger (burger) : shami bun kabob non veg
burger (burger) : hamburger with meal
burger (burger) : pride burger
burger (burger) : whisky bacon burger
burger (burger) : cesar burger
burger (burger) : double cheeseburger
burger (burger) : hamburg

salad (salad) : ensalada de la casa
salad (salad) : baby gems
salad (salad) : anarkali salad vegan
salad (salad) : meatball marinara
salad (salad) : blackened shrimp caesar salad
salad (salad) : caesar salad
salad (salad) : beet salad
salad (salad) : tossed salad
salad (salad) : ensalada de mariscos
salad (salad) : cabo chop
salad (salad) : steak salad
salad (salad) : chicken caesar
salad (salad) : misto salad with chicken
salad (salad) : insalata mista piccolo
salad (salad) : loaded gyro salad
salad (salad) : greek salad
salad (salad) : shrimp salad
salad (salad) : taco salad
salad (salad) : garden salad
salad (salad) : chicken caesar salad
salad (salad) : caesar salad
salad (salad) : sweet n heat goes greek salad combo
salad (salad) : avocado mango salad with salmon
salad (salad) : sm raspberry chipotle chicken salad
salad (salad) : chicken salad
salad (salad) : chicken salad
salad (salad) : house salad
salad (salad) : house salad
salad (salad) : havana salad
salad (salad) : larb
sal

sandwich (sandwich) : turkey
sandwich (sandwich) : deluxe b l t a
sandwich (sandwich) : spring chicken wrap
sandwich (sandwich) : michael jordan
sandwich (sandwich) : breakfast sandwich
sandwich (sandwich) : the famous reuben
sandwich (sandwich) : blt sandwich
sandwich (sandwich) : bbq chicken
sandwich (sandwich) : tuna melt
sandwich (sandwich) : bbq melt
sandwich (sandwich) : big oak steak burger
sandwich (sandwich) : smoked salmon panini
sandwich (sandwich) : catering chicken caesar wrap
sandwich (sandwich) : pollock fish
sandwich (sandwich) : hot open faced sandwich
sandwich (sandwich) : french dip
sandwich (sandwich) : california turkey
sandwich (sandwich) : grilled breast of chicken sandwich deluxe
sandwich (sandwich) : breaskfast sandwich
sandwich (sandwich) : smoked gouda burger
sandwich (sandwich) : chopped chicken
sandwich (sandwich) : turkey bacon wrap
sandwich (sandwich) : chicken salad
sandwich (sandwich) : corned beef
sandwich (sandwich) : chicken blat
sandwich (sandwich) 

burger (burger) : triple whopper with cheese
burger (burger) : tamale burger
burger (burger) : breakfast burger
burger (burger) : barbecue cheeseburger
burger (burger) : hamburger with french fries
burger (burger) : jalapenos burger
burger (burger) : spicystacker
burger (burger) : cheeseburger
burger (burger) : cheeseburger
burger (burger) : turkey burger
burger (burger) : bacon all american cheeseburger
burger (burger) : chili cheese burger
burger (burger) : the mariano rivera
burger (burger) : new york
burger (burger) : the outlaw
burger (burger) : the lyric
burger (burger) : big king xl
burger (burger) : italian burger
burger (burger) : the ultra smurf
burger (burger) : chili cheese burger
burger (burger) : all american cheeseburger double
burger (burger) : bacon burger
burger (burger) : double all american
burger (burger) : standard
burger (burger) : whisky bourbon burger
burger (burger) : turkey burger
=====
